# Download and unzip the PascalVoc dataset

In [1]:
import torchvision

data_dir = './voc'

train_dataset = torchvision.datasets.VOCSegmentation(root=data_dir, year='2007', download=False, image_set='train')
val_dataset = torchvision.datasets.VOCSegmentation(root=data_dir, year='2007', download=False, image_set='val')
test_dataset = torchvision.datasets.VOCSegmentation(root=data_dir, year='2007', download=False, image_set='test')



In [2]:
print(f"Train set size: {len(train_dataset)}")
print(f"Val set size: {len(val_dataset)}")
print(f"Test set size: {len(test_dataset)}")


Train set size: 209
Val set size: 213
Test set size: 210


# Data Preparation


run the code below to get thre dataloader objects, namely: train_loader, val_loader and test_loader

In [3]:
import os
from PIL import Image
from torch.utils import data
import torchvision.transforms as transforms
import random

num_classes = 21
ignore_label = 255
root = data_dir

'''
color map
0=background, 1=aeroplane, 2=bicycle, 3=bird, 4=boat, 5=bottle # 6=bus, 7=car, 8=cat, 9=chair, 10=cow, 11=diningtable,
12=dog, 13=horse, 14=motorbike, 15=person # 16=potted plant, 17=sheep, 18=sofa, 19=train, 20=tv/monitor
'''


#Feel free to convert this palette to a map
palette = [0, 0, 0, 128, 0, 0, 0, 128, 0, 128, 128, 0, 0, 0, 128, 128, 0, 128, 0, 128, 128,
           128, 128, 128, 64, 0, 0, 192, 0, 0, 64, 128, 0, 192, 128, 0, 64, 0, 128, 192, 0, 128,
           64, 128, 128, 192, 128, 128, 0, 64, 0, 128, 64, 0, 0, 192, 0, 128, 192, 0, 0, 64, 128]  #3 values- R,G,B for every class. First 3 values for class 0, next 3 for
#class 1 and so on......

'''
Depending on the mode, train or val or test, the function reads the train.txt, val.txt and test.txt files and returns a list of tuples of the form
(image_path, mask_path) for each image in the dataset, where image_path is the path to the image and mask_path is the path to the mask for that image. 
'''
def make_dataset(mode):
    
    assert mode in ['train', 'val', 'test', 'trainval']
    
    items = []
    img_path = os.path.join(root, 'VOCdevkit', 'VOC2007', 'JPEGImages')
    mask_path = os.path.join(root, 'VOCdevkit', 'VOC2007', 'SegmentationClass')
    data_list = [l.strip('\n') for l in open(os.path.join(
        root, 'VOCdevkit', 'VOC2007', 'ImageSets', 'Segmentation', f'{mode}.txt')).readlines()]
    for it in data_list:
        item = (os.path.join(img_path, it + '.jpg'), os.path.join(mask_path, it + '.png'))
        items.append(item)

        
    return items


'''
The class VOC is a subclass of the class torch.utils.data.Dataset. It overrides the __len__ and __getitem__ methods.
The __len__ method returns the length of the dataset, i.e. the number of images in the dataset.
The __getitem__ method returns the image and the mask for the given index.
'''

class VOC(data.Dataset):
    def __init__(self, mode, transform=None, target_transform=None, common_transform=None):
        self.imgs = make_dataset(mode)        
        if len(self.imgs) == 0:
            raise RuntimeError('Found 0 images, please check the data set')
        self.mode = mode
        self.transform = transform
        self.target_transform = target_transform
        self.common_transform = common_transform
        self.width = 256
        self.height = 256

    def __getitem__(self, index):
        
        img_path, mask_path = self.imgs[index]
        img = Image.open(img_path).convert('RGB').resize((self.width, self.height))
        mask = Image.open(mask_path).resize((self.width, self.height))

        if self.common_transform is not None:
            img, mask = self.common_transform((img,mask)) 

        if self.transform is not None:
            img = self.transform(img)
        if self.target_transform is not None:
            mask = self.target_transform(mask)

        mask[mask==ignore_label]=0

        return img, mask

    def __len__(self):
        
        return len(self.imgs)

In [4]:
import numpy as np 
import torch
class MaskToTensor(object):
    def __call__(self, img):
        return torch.from_numpy(np.array(img, dtype=np.int32)).long()


mean_std = ([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])

input_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(*mean_std)
])

target_transform = transforms.Compose([
    MaskToTensor()
])

original_train_dataset = VOC('trainval', transform=input_transform, target_transform=target_transform)
# original_val_dataset = VOC('val', transform=input_transform, target_transform=target_transform)
original_test_dataset = VOC('test', transform=input_transform, target_transform=target_transform)


In [5]:
from torch.utils.data import DataLoader, ConcatDataset

NUM_WORKERS = 2
PREFETCH_FACTOR = 2 # improves data transfer speed between GPU and CPU and reduces GPU wait time
train_loader = DataLoader(dataset=original_train_dataset, batch_size=16, shuffle=True, num_workers=NUM_WORKERS, prefetch_factor=PREFETCH_FACTOR, pin_memory=True)

# val_loader = DataLoader(dataset=original_val_dataset, batch_size=16, shuffle=False, num_workers=NUM_WORKERS, prefetch_factor=PREFETCH_FACTOR, pin_memory=True)

test_loader = DataLoader(dataset=original_test_dataset, batch_size= 16, shuffle=False, num_workers=NUM_WORKERS, prefetch_factor=PREFETCH_FACTOR, pin_memory=True)


In [6]:
# from torchvision.datasets import Cityscapes

# data_dir = "./data/cityscapes"

# train_dataset = Cityscapes(root=data_dir, split="train", mode="fine", target_type="semantic", transform=input_transform, target_transform=target_transform)

# utils 

In [7]:
from copy import deepcopy
def iou(pred, target, n_classes = 21):
    target[target==255] = 0

    ious = []

    for cls in range(n_classes):
        intersection = torch.sum((pred == cls) & (target == cls)).item()
        union = torch.sum(pred == cls) + torch.sum(target == cls) - intersection
        union = union.item()
        if union!=0:
            ious.append(intersection/union)

    ious = np.array(ious)
    return ious

'''
returns pixel accuracy for the batch
'''
def pixel_acc(pred, target):
    target[target==255] = 0
    
    correct = torch.sum(pred==target).item()
    total_predictions = target.shape[0]*target.shape[1]*target.shape[2]
    return correct/total_predictions

In [8]:
import time
import torch.nn.functional as F

criterion = torch.nn.CrossEntropyLoss()

def train(model=None):

    model_ = model 
    torch.autograd.set_detect_anomaly(True)
    
    best_iou_score = 0.0

    trainEpochLoss = []
    trainEpochAccuracy = []
    trainEpochIOU = []
    valEpochLoss = []
    valEpochAccuracy = []
    valEpochIOU = []

    for epoch in range(epochs):

        # with profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA], record_shapes=True) as prof:
        train_loss = []
        train_acc = []
        train_iou = []

        ts = time.time()
        for itr, (inputs, labels) in enumerate(train_loader):
            #   reset optimizer gradients
            optimizer.zero_grad()

            inputs =  inputs.to(device)
            labels =   labels.to(device)

            trainOutputs =  model_(inputs)
#             trainOutputs = F.softmax(trainOutputs, dim=1)
            loss =  criterion(trainOutputs,labels)  #  calculate loss
            loss.backward()

            with torch.no_grad():
                # To compute accuracy and IOU
                # outputs = F.log_softmax(model_(inputs), dim=1)
                _, pred = torch.max(trainOutputs, dim=1)
                
                train_iou.append(np.mean(iou(pred, labels)))
                train_acc.append(pixel_acc(pred, labels))
                train_loss.append(loss.item())

            optimizer.step()

            if itr % 10 == 0:
                print(f"==> epoch{epoch}, iter{itr+1}, Train set=> loss: {np.mean(train_loss)}, IOU: {np.mean(train_iou)}, Acc: {np.mean(train_acc)}")

        # print(prof.key_averages().table(sort_by="cpu_time_total", row_limit=10))

        print(f"Finish epoch {epoch}, time elapsed {time.time() - ts}")

        val_loss, val_iou, val_acc = test(epoch,model_)
        model_.train()


In [9]:
def test(epoch, model=None):
    model_ = model
    model_.eval() # Put in eval mode (disables batchnorm/dropout) !
    
    losses = []
    mean_iou_scores = []
    accuracy = []

    with torch.no_grad(): # we don't need to calculate the gradient in the validation/testing
        num_iter = 0
        for iter, (inputs, labels) in enumerate(test_loader):
            
            # both inputs and labels have to reside in the same device as the model's
            inputs =  inputs.to(device)#  transfer the input to the same device as the model's
            labels =   labels.to(device)#  transfer the labels to the same device as the model's


            outputs = model_(inputs)
            outputs = F.softmax(outputs, dim=1)
#             valoutputs = model_(inputs)
            valloss = criterion(outputs, labels)
            
            num_iter += 1
            _, pred = torch.max(outputs, dim=1)
            
            mean_iou_scores.append(np.mean(iou(pred, labels)))
            accuracy.append(pixel_acc(pred, labels))
            losses.append(valloss.item())

    # print(mean_iou_scores, accuracy)
    print(f"=========> Loss at epoch {epoch} is {np.mean(losses)}")
    print(f"=========> IoU at epoch {epoch} is {np.mean(mean_iou_scores)}")
    print(f"=========> Pixel acc at epoch {epoch} is {np.mean(accuracy)}")

    
    return np.mean(losses), np.mean(mean_iou_scores), np.mean(accuracy)

# SSL models

In [10]:
import torch.nn as nn


class Backbone(nn.Module):
    def __init__(self, in_chan, out_dim):
        super(Backbone, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=in_chan, out_channels=16, kernel_size=3, stride=1)
        self.bn1 = nn.BatchNorm2d(16)
        self.relu1 = nn.ReLU()
        
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1)
        self.bn2 = nn.BatchNorm2d(32)
        self.relu2 = nn.ReLU()
        
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1)
        self.bn3 = nn.BatchNorm2d(64)
        self.relu3 = nn.ReLU()
        
        self.conv4 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.bn4 = nn.BatchNorm2d(128)
        self.relu4 = nn.ReLU()
        self.maxpool4 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.conv5 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1)
        self.bn5 = nn.BatchNorm2d(256)
        self.relu5 = nn.ReLU()
        self.maxpool5 = nn.MaxPool2d(kernel_size=2, stride=2)        
        
        self.pooling = nn.AdaptiveAvgPool2d((1, 1))
        
        self.fc = nn.Linear(in_features=256, out_features=out_dim)
        self.relu6 = nn.ReLU()
        
        self.classifier = nn.Linear(out_dim, 20)
    
    def forward(self, x):
        x = self.relu1(self.bn1(self.conv1(x)))
        x = self.relu2(self.bn2(self.conv2(x)))
        x = self.relu3(self.bn3(self.conv3(x)))

        x = self.relu4(self.bn4(self.conv4(x)))
        x = self.maxpool4(x)
        
        x = self.relu5(self.bn5(self.conv5(x)))
        x = self.maxpool5(x)

        x = self.pooling(x)
#         embed = self.fc(x.squeeze())
        
#         x = self.relu6(embed)
#         x = self.classifier(x)
        
        return x

In [11]:
import torch
import torch.nn as nn

class Segmentor(nn.Module):
    def __init__(self, n_class=21, n_dim=512):
        super(Segmentor, self).__init__()

        # Encoder (Based on the provided SSL architecture)
        self.encoder = Backbone(in_chan=3, out_dim=n_dim)
        
#         self.reducer = nn.Conv2d(256, 8, kernel_size=1)
        
        # do some drastic upsampling from (1,1) to (16, 16) to reduce overall parameters
        self.drastic = nn.ConvTranspose2d(256, 256, kernel_size=(16,16), stride = 1)
        
        # Decoder
        self.decoder4 = self.expanding_block(256, 64)
#         self.decoder3 = self.expanding_block(128, 64)
        self.decoder2 = self.expanding_block(64, 32)
#         self.decoder1 = self.expanding_block(32, 16)
        
        # Output layer
        self.output = nn.Conv2d(32, n_class, kernel_size=1)
        
    def expanding_block(self, in_channels, out_channels):
        block = nn.Sequential(
            nn.ConvTranspose2d(in_channels, out_channels, kernel_size=2, stride=2),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(out_channels, out_channels, kernel_size=2, stride=2),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
        )
        return block
    
    def forward(self, x):
        # Encoder (SSL model)
        features = self.encoder(x)
        
#         features = self.reducer(features)
        features = self.drastic(features)
        
        # Decoder
        decode4 = self.decoder4(features)
#         decode3 = self.decoder3(decode4)
        decode2 = self.decoder2(decode4)
    
#         decode1 = self.decoder1(decode2)
        
        # Output
        output = self.output(decode2)
        output = nn.ReLU(inplace=True)(output)
        
        return output


In [12]:
from torch import optim

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Assuming 21 classes for segmentation
num_classes = 21

# Create the overall segmentor model
model = Segmentor(n_class=num_classes, n_dim=512)

In [13]:
# Load the SSL model weights
# backbone_weights = torch.load('pet_data_dc_e20.pth', map_location=torch.device(device))
# model.encoder.load_state_dict(backbone_weights)

# # freeze the backbone
# for param in model.encoder.parameters():
#     param.requires_grad = False

optimizer = optim.Adam(model.parameters(), lr=0.0025)
model = model.to(device)


In [ ]:
epochs = 20
train(model)


==> epoch0, iter1, Train set=> loss: 2.9825336933135986, IOU: 0.008565173917214737, Acc: 0.055332183837890625
==> epoch0, iter11, Train set=> loss: 2.638412670655684, IOU: 0.027129403039298505, Acc: 0.4146718111905185
==> epoch0, iter21, Train set=> loss: 2.4523055099305653, IOU: 0.03542650354071118, Acc: 0.486954234895252
Finish epoch 0, time elapsed 9.29441237449646
=========> Loss at epoch 0 is 2.8679775340216502
=========> IoU at epoch 0 is 0.04805766794990152
=========> Pixel acc at epoch 0 is 0.550309453691755
==> epoch1, iter1, Train set=> loss: 1.966254472732544, IOU: 0.051775299316090935, Acc: 0.5930366516113281
==> epoch1, iter11, Train set=> loss: 1.908684578808871, IOU: 0.0432814828988641, Acc: 0.5786150152033026
==> epoch1, iter21, Train set=> loss: 1.8871495156061082, IOU: 0.042558514593192376, Acc: 0.5656367710658482
Finish epoch 1, time elapsed 7.2943689823150635
=========> Loss at epoch 1 is 2.7081760338374545
=========> IoU at epoch 1 is 0.04808073280926545
=========>

=========> Loss at epoch 15 is 2.517987404550825
=========> IoU at epoch 15 is 0.05576357488509349
=========> Pixel acc at epoch 15 is 0.7290957314627511
==> epoch16, iter1, Train set=> loss: 1.3903385400772095, IOU: 0.0451473871866862, Acc: 0.677210807800293
==> epoch16, iter11, Train set=> loss: 1.2385356426239014, IOU: 0.05039929832964485, Acc: 0.7242961363358931
==> epoch16, iter21, Train set=> loss: 1.1717378837721688, IOU: 0.052043365292604594, Acc: 0.7432876768566313
Finish epoch 16, time elapsed 7.358681678771973
=========> Loss at epoch 16 is 2.5337564774921963
=========> IoU at epoch 16 is 0.05576357488509349
=========> Pixel acc at epoch 16 is 0.7290957314627511
==> epoch17, iter1, Train set=> loss: 1.2266993522644043, IOU: 0.0562292979313777, Acc: 0.7309808731079102
==> epoch17, iter11, Train set=> loss: 1.098292583769018, IOU: 0.05465734555661457, Acc: 0.7604439475319602
==> epoch17, iter21, Train set=> loss: 1.177521146479107, IOU: 0.05242506798362262, Acc: 0.745144435337

In [14]:
# unfreeze the backbone
for param in model.encoder.parameters():
    param.requires_grad = True

    
# train further
epochs = 20

train(model)


==> epoch0, iter1, Train set=> loss: 3.0092945098876953, IOU: 0.00622066843993406, Acc: 0.03785419464111328
==> epoch0, iter11, Train set=> loss: 2.7170171304182573, IOU: 0.02717147469512641, Acc: 0.3603216518055309
==> epoch0, iter21, Train set=> loss: 2.57073712348938, IOU: 0.03393305857502766, Acc: 0.4372586295718238
Finish epoch 0, time elapsed 9.195792198181152
=========> Loss at epoch 0 is 3.0226228748049055
=========> IoU at epoch 0 is 0.04670802049448038
=========> Pixel acc at epoch 0 is 0.6369955199105399
==> epoch1, iter1, Train set=> loss: 1.9593805074691772, IOU: 0.03935642219430873, Acc: 0.6270961761474609
==> epoch1, iter11, Train set=> loss: 2.000320141965693, IOU: 0.043674859470311585, Acc: 0.5713292035189542
==> epoch1, iter21, Train set=> loss: 1.9313550563085646, IOU: 0.04415074929216892, Acc: 0.5687716801961263
Finish epoch 1, time elapsed 7.29560661315918
=========> Loss at epoch 1 is 2.8765545231955394
=========> IoU at epoch 1 is 0.04590464831053393
=========> P

KeyboardInterrupt: 